In [3]:
import pandas as pd
import numpy as np
import pydataset
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

In [4]:
np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [6]:
df = spark.createDataFrame(pandas_dataframe)

In [8]:
df.show()

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
|  5|    c|
|  6|    a|
|  7|    b|
|  8|    a|
|  9|    b|
| 10|    b|
| 11|    a|
| 12|    b|
| 13|    a|
| 14|    b|
| 15|    b|
| 16|    c|
| 17|    c|
| 18|    a|
| 19|    c|
+---+-----+



In [15]:
df[['group', 'n']]

DataFrame[group: string, n: bigint]

In [14]:
df['group']

Column<b'group'>

In [13]:
df.select(df.group).show()

+-----+
|group|
+-----+
|    b|
|    b|
|    c|
|    a|
|    c|
|    c|
|    a|
|    b|
|    a|
|    b|
|    b|
|    a|
|    b|
|    a|
|    b|
|    b|
|    c|
|    c|
|    a|
|    c|
+-----+



In [19]:
n_incremented = df.n + 1

In [21]:
n_incremented

Column<b'(n + 1)'>

In [20]:
df.select(n_incremented).show()

+-------+
|(n + 1)|
+-------+
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
|     10|
|     11|
|     12|
|     13|
|     14|
|     15|
|     16|
|     17|
|     18|
|     19|
|     20|
+-------+



In [23]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



In [36]:
pd.read_csv('alksdhbfljashbdfjhasd')

FileNotFoundError: [Errno 2] File b'alksdhbfljashbdfjhasd' does not exist: b'alksdhbfljashbdfjhasd'

In [34]:
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



In [33]:
df.describe().select('n', 'summary').show()

+-----------------+-------+
|                n|summary|
+-----------------+-------+
|               20|  count|
|              9.5|   mean|
|5.916079783099616| stddev|
|                0|    min|
|               19|    max|
+-----------------+-------+



In [35]:
df.select('n', 'summary').describe().show()

AnalysisException: "cannot resolve '`summary`' given input columns: [n, group];;\n'Project [n#4L, 'summary]\n+- LogicalRDD [n#4L, group#5], false\n"

In [41]:
mpg = spark.createDataFrame(pydataset.data('mpg'))
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [45]:
mpg.hwy

Column<b'hwy'>

In [46]:
mpg.select(mpg.model, 'manufacturer', mpg.hwy.alias('highway_mileage')).show()

+------------------+------------+---------------+
|             model|manufacturer|highway_mileage|
+------------------+------------+---------------+
|                a4|        audi|             29|
|                a4|        audi|             29|
|                a4|        audi|             31|
|                a4|        audi|             30|
|                a4|        audi|             26|
|                a4|        audi|             26|
|                a4|        audi|             27|
|        a4 quattro|        audi|             26|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             28|
|        a4 quattro|        audi|             27|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             25|
|        a6 quattro|        audi|             24|
|        a6 quattro|        audi|             25|


In [47]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [55]:
avg_mileage_column = ((mpg.cty + mpg.hwy) / 2).alias('avg_mileage')
avg_mileage_column

Column<b'((cty + hwy) / 2) AS `avg_mileage`'>

In [56]:
mpg.select('*', avg_mileage_column).show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+-----------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|avg_mileage|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+-----------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|       23.5|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|       25.0|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|       25.5|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|       25.5|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|       21.0|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|       22.0|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|       22.5|
|        audi|        a4 quattro|  1.8|1

In [58]:
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [60]:
from pyspark.sql.functions import col

col

<function pyspark.sql.functions._create_function.<locals>._(col)>

In [66]:
col('abc')

Column<b'abc'>

In [65]:
col('hwy') * 2

Column<b'(hwy * 2)'>

In [72]:
just_hwy_and_cty = mpg.select('hwy', 'cty').show(5)

just_hwy_and_cty.select('hwy').show()

+---+---+
|hwy|cty|
+---+---+
| 29| 18|
| 29| 21|
| 31| 20|
| 30| 21|
| 26| 16|
+---+---+
only showing top 5 rows



AttributeError: 'NoneType' object has no attribute 'select'

In [74]:
mpg.select(col('hwy').alias('highway_mileage')).show(5)

+---------------+
|highway_mileage|
+---------------+
|             29|
|             29|
|             31|
|             30|
|             26|
+---------------+
only showing top 5 rows



In [75]:
from pyspark.sql.functions import expr

In [77]:
mpg.select(expr('hwy AS highway_mileage')).show(5)

+---------------+
|highway_mileage|
+---------------+
|             29|
|             29|
|             31|
|             30|
|             26|
+---------------+
only showing top 5 rows



In [78]:
mpg.select(expr('(hwy + cty) / 2 AS average_mileage')).show(5)

+---------------+
|average_mileage|
+---------------+
|           23.5|
|           25.0|
|           25.5|
|           25.5|
|           21.0|
+---------------+
only showing top 5 rows



In [81]:
mpg.select(expr('AVG(hwy)')).show()

+-----------------+
|         avg(hwy)|
+-----------------+
|23.44017094017094|
+-----------------+



In [82]:
mpg.createOrReplaceTempView('mpg')

In [87]:
mpg2 = spark.sql('''
SELECT
    cty AS city,
    hwy AS highway,
    (cty + hwy) / 2 AS avg_mileage
FROM mpg
WHERE class = "compact"
''')

In [88]:
mpg2

DataFrame[city: bigint, highway: bigint, avg_mileage: double]

In [89]:
mpg2.select('city', 'highway')

DataFrame[city: bigint, highway: bigint]

In [91]:
mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [92]:
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [93]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [98]:
mpg.select('*', mpg.cyl.cast('string').alias('cyl_string')).show(4)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|cyl_string|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|         4|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|         4|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|         4|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|         4|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+
only showing top 4 rows



In [99]:
spark.sql('SELECT *, CAST(cyl AS STRING) AS cyl_string FROM mpg').show(4)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|cyl_string|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|         4|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|         4|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|         4|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|         4|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+
only showing top 4 rows



In [103]:
mpg.select(mpg.manufacturer.cast('double')).show()

+------------+
|manufacturer|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+
only showing top 20 rows



In [105]:
min([1, 2, 3])

1

In [106]:
from pyspark.sql.functions import min, max

In [107]:
min([1, 2, 3])

Py4JError: An error occurred while calling z:org.apache.spark.sql.functions.min. Trace:
py4j.Py4JException: Method min([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [111]:
mpg.select(min(mpg.cyl).alias('min_cyl'), max('hwy')).show()

+-------+--------+
|min_cyl|max(hwy)|
+-------+--------+
|      4|      44|
+-------+--------+



In [112]:
spark.sql('SELECT min(cyl), max(hwy) FROM mpg').show()

+--------+--------+
|min(cyl)|max(hwy)|
+--------+--------+
|       4|      44|
+--------+--------+

